## TS2 - Problem wieloagentowy - Paweł Malisz, Mikołaj Białek - Rock Paper Scissors

### 1. Opis gry

Papier Kamień Nożyce to jedna z najstarszych dwuosobowych gier ręcznych na świecie. Gracz stara się przewidzieć ruch przeciwnika wnioskując na jego poprzednich ruchach i wygrać za pomocą jednego z elementów, który wybiera na tzn. (raz, dwa, trzy!). W standardowej wersją są trzy elementy: 
 - kamień - wygrywający z nożyczkami,
 - papier - wygrywający z kamienień,
 - nożyczki - wygrywające z papierem.

Gre można rozszerzyć o dodatkowe elementy takie jak np. jaszczurkę i spock, wtedy mamy wariant 5-elementowy. (zawsze musi być to liczba nieparzysta, czyli 3, 5, 7, 9 itd.)

### 2. Cel ćwiczenia

Celem naszego ćwiczenia jest rozwiązanie problemu wieloagentowego z biblioteki PettingZoo używając uczenia mszynowego w języku Python. 

Przedstawiony zostanie algorytm DQN (Deep Q-learning) na platformie Tianshou wraz z krzywą uczenia.

### 3. Opis problemu

Na grę składają się dwa dyskretne zbiory akcji i obserwacji. Zbiór akcji zawiera ilość dostępnych ruchów dla gracza (w standardowej wersji są to 3 ruchy). Natomiast zbiór obserwacji składa się z n + 1 ruchów (czyli 4 dla standardowej wersji gry), ostatnią akcją jest tzn. obserwacja, czyli moment w którym gracz jeszcze nic nie wybrał. Chcielibyśmy nauczyć naszego agenta, aby podejmował intuicyjną decyzję bazując na poprzednich ruchach swojego przeciwnika.

### 4. Rozwiązanie problemu

Zacznijmy od funkcji definiującej nasze środowisko i po krótcę ją opiszmy.

Nasze klasyczne środowisko zawiera dwóch agentów oraz poniższe możliwe opcje:
- `num_actions` - ilość dostępnych ruchów (elementów) jako liczba nieparzysta,
- `max_cycles` - ilośc cykli po jakiej nasi agenci zakończą grę.

In [ ]:
from tianshou.env.pettingzoo_env import PettingZooEnv
from pettingzoo.classic import rps_v2

def _get_env():
    return PettingZooEnv(rps_v2.env(num_actions=3, max_cycles=15))

Stwórzmy nasze środowisko, 100 próbek uczących i 10 testowych, musimy przekształcić na obiekt `DummyVectorEnv`.

In [ ]:
from tianshou.env import DummyVectorEnv

train_envs = DummyVectorEnv([_get_env for _ in range(100)])
test_envs = DummyVectorEnv([_get_env for _ in range(10)])

Teraz stwórzmy funkcję dla naszych agentów, zrobimy tak, by `player_1` był naszym uczeniem maszynowym, a `player_0` podejmował losowe ruchy.

Opis kodu:
- `player_0` = `agent_opponent` - struktura klasy `RandomPolicy()`,
- `player_1` = `agent_learn` - struktura klasy `DQNPolicy()`, która przyjmuje takie wartości jak:
    - `model` - struktura w tym przypadku klasy `Net()`, (chociaż mogłaby być to też zapewne `Actor()` z `tianshou.utils.net.discrete`), opisuje nam stosowaną sieć neuronową, domyślnie z funkcją aktywacji ReLu,
    - `optim` - czyli optymalizator, domnyslnie sotuje się Adam, więc taki zostawiamy,
    - `discount_factor`,
    - `estimation_step` - przewidywana ilość ruchów (w naszym przypadku 1),
    - `target_update_freq` - określa częstotliwość aktualizacji tzw. target network (sieci docelowej) w stosunku do policy network (sieci polityki).

Później łączymy dwie polityki w jedną `MultiAgentPolicyManager()`.

In [ ]:
import torch

from typing import Optional, Tuple
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy, PPOPolicy
from tianshou.utils.net.common import Net, ActorCritic
from tianshou.utils.net.discrete import Actor, Critic

def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    if agent_learn is None:
        # model
        device = "cuda" if torch.cuda.is_available() else "cpu"
        net = Net(
            state_shape=env.observation_space.shape or env.observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[64, 64],
            device=device,
        ).to(device)
        actor = Actor(net, env.action_space.n, device=device).to(device)
        critic = Critic(net, device=device).to(device)
        actor_critic = ActorCritic(actor, critic)

        if optim is None:
            optim = torch.optim.Adam(actor_critic.parameters(), lr=0.0003)
            dist = torch.distributions.Categorical
            agent_learn = PPOPolicy(actor, critic, optim, dist, action_space=env.action_space.shape or env.action_space.n, deterministic_eval=True)

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    print(agents)
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents

Następnie inicjalizujemy naszych agentów, oraz ustawiamy Collectory, czyli obiekt `Collector` z biblioteki `Tianshou`. Służy onm do zbierania danych treningowych oraz testowych w procesie uczenia ze wzmocnieniem. Służy do interakcji z środowiskiem, gromadzenia trajektorii agentów oraz przechowywania tych trajektorii w pamięci podręcznej w celu późniejszego wykorzystania w procesie uczenia.

In [ ]:
from tianshou.data import Collector, VectorReplayBuffer

policy, optim, agents = _get_agents()
policy.train()

train_collector = Collector(policy, train_envs, VectorReplayBuffer(50000, len(train_envs)))
test_collector = Collector(policy, test_envs)

train_collector.reset()
train_collector.collect(n_episode=1)

Teraz etap uczenia, służy do niego funkcja `offpolicy_trainer()` z biblioteki `tianshou`.

In [ ]:
from tianshou.trainer import offpolicy_trainer
import tianshou as ts

from torch.utils.tensorboard import SummaryWriter
import os

lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 10
logger = ts.utils.TensorboardLogger(SummaryWriter('log/dqn'))

def save_best_fn(policy):
    model_save_path = os.path.join("log", "rps", "dqn", "policy.pth")
    os.makedirs(os.path.join("log", "rps", "dqn"), exist_ok=True)
    torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

def stop_fn(mean_rewards):
    return mean_rewards >= 0.6

def train_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.1)

def test_fn(epoch, env_step):
    policy.policies[agents[1]].set_eps(0.05)

def reward_metric(rews):
    return rews[:, 1]

result = offpolicy_trainer(
    policy=policy,
    train_collector=train_collector,
    test_collector=test_collector,
    max_epoch=50,
    step_per_epoch=1000,
    step_per_collect=50,
    episode_per_test=10,
    batch_size=64,
    train_fn=train_fn,
    test_fn=test_fn,
    stop_fn=stop_fn,
    save_best_fn=save_best_fn,
    update_per_step=0.1,
    test_in_train=False,
    reward_metric=reward_metric,
    logger=logger
)
print(f'Finished training! Use {result["duration"]}')

In [ ]:
from tianshou.trainer import onpolicy_trainer

result = onpolicy_trainer(
    policy,
    train_collector,
    test_collector,
    max_epoch=10,
    step_per_epoch=50000,
    repeat_per_collect=10,
    episode_per_test=10,
    batch_size=256,
    step_per_collect=2000,
    stop_fn=lambda mean_reward: mean_reward >= 195,
)

Zapisanie wytrenowanej polityki

In [ ]:
torch.save(policy.state_dict(), 'dqn.pth')
policy.load_state_dict(torch.load('dqn.pth'))

Wytestowanie naszego uczenia

In [ ]:
policy.eval()
collector = ts.data.Collector(policy, _get_env(), exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

In [ ]:
!tensorboard --logdir log